# [AdventOfCode](http://adventofcode.com/) 2015 solutions, a single line of bash per challenge

This document is a [jupyter notebook](https://jupyter.org), to run it interactively you will need the [bash_kernel](https://github.com/takluyver/bash_kernel), but I don't recommend running it, as it chokes on commands with huge output.

While the code presented here is neither efficient nor readable, it was fun to write.

### Preparations

I will need some way to read challange input data, `d N` simply reads `data/N` and dumps its contents to stdout. Some data files are quite big, so I wrote `h` to read only the first few bytes/lines of the input files. Not every challenge is included here.

In [56]:
d() { 
    cat "data/${1:?Missing chall num}"
}
h() { 
    local linesize="$(d "$1" | awk '{print length($0); exit}')"
    if test "$linesize" -gt 100
    then d "$1" | head -c 40
    else d "$1" | head -n 10
    fi
}

## Day 1: Not Quite Lisp 

### part1


> Santa was hoping for a white Christmas, but his weather machine's "snow" function is powered by stars, and he's fresh out!
> To save Christmas, he needs you to collect fifty stars by December 25th.
> 
> Collect stars by helping Santa solve puzzles.
> Two puzzles will be made available on each day in the advent calendar; the second puzzle is unlocked when you complete the first.
> Each puzzle grants one star. Good luck!
> 
> Here's an easy puzzle to warm you up.
> 
> Santa is trying to deliver presents in a large apartment building, but he can't find the right floor - the directions he got are a little confusing.
> He starts on the ground floor (floor 0) and then follows the instructions one character at a time.
> 
> An opening parenthesis, `(`, means he should go up one floor, and a closing parenthesis, `)`, means he should go down one floor.
> 
> The apartment building is very tall, and the basement is very deep; he will never find the top or bottom floors.
> 
> For example:
> 
> - `(())` and `()()` both result in floor 0.
> 
> - `(((` and `(()(()(` both result in floor 3.
> 
> - `))(((((` also results in floor 3.
> 
> - `())` and `))(` both result in floor -1 (the first basement level).
> 
> - `)))` and `)())())` both result in floor -3.
> 
> To what floor do the instructions take Santa?

Lets see the input data first:

In [57]:
h 1

()()(()()()(()()((()((()))((()((((()()((

Ok, so the challange is to count the two type of parenthesis and subtract them from each other. We could use `grep -c` or `sort | uniq -c` to count, but these methods are somewhat clumsy, so I went with a different approach: convert every `(` to `+1` and every `)` to `-1` then calculate the resulting expression. Lets do the conversion first, these lines all do the same

In [58]:
h 1 | tr '()' +- | sed 's/./\01/g'
echo
h 1 | sed 'y/()/+-/;s/./\01/g'
echo
h 1 | sed 's/(/+1/g;s/)/-1/g'

+1-1+1-1+1+1-1+1-1+1-1+1+1-1+1-1+1+1+1-1+1+1+1-1-1-1+1+1+1-1+1+1+1+1+1-1+1-1+1+1
+1-1+1-1+1+1-1+1-1+1-1+1+1-1+1-1+1+1+1-1+1+1+1-1-1-1+1+1+1-1+1+1+1+1+1-1+1-1+1+1
+1-1+1-1+1+1-1+1-1+1-1+1+1-1+1-1+1+1+1-1+1+1+1-1-1-1+1+1+1-1+1+1+1+1+1-1+1-1+1+1

The binary calculator `bc` can be used to evaluate this expression:

In [59]:
h 1 | sed 's/(/+1/g;s/)/-1/g' | bc

(standard_in) 1: syntax error
(standard_in) 1: syntax error


Hmmm, `bc` chokes on the leading `+` sign. We can solve it by prepending  `0`. This can be done with `cat` and process substitution, or with the handy "here string" notation.

In [60]:
d 1 | cat <(printf 0) - | sed 's/(/+1/g;s/)/-1/g' | bc
bc<<<0$(d 1|sed 's/(/+1/g;s/)/-1/g')

280
280


### part2

> Now, given the same instructions, find the position of the first character that causes him to enter the basement (floor -1).
> The first character in the instructions has position 1, the second character has position 2, and so on.
> 
> For example:
> 
> - `)` causes him to enter the basement at character position 1.
> 
> - `()())` causes him to enter the basement at character position 5.
> 
> **What is the position of the character that causes Santa to first enter the basement?**

I used `awk`, as any sane person would. `fold -w1` splits its input into 1-length lines.

In [61]:
d 1|fold -w1|awk '{s+=$1=="("?1:-1;if(s<0){print NR;exit}}'

1797


## Day 2: I Was Told There Would Be No Math

### part1

> The elves are running low on wrapping paper, and so they need to submit an order for more.
> They have a list of the dimensions (length `l`, width `w`, and height `h`) of each present, and only want to order exactly as much as they need.
> 
> Fortunately, every present is a box (a perfect right rectangular prism), which makes calculating the required wrapping paper for each gift a little easier: find the surface area of the box, which is `2*l*w + 2*w*h + 2*h*l`.
> The elves also need a little extra paper for each present: the area of the smallest side.
> 
> For example:
> 
> - A present with dimensions `2x3x4` requires `2*6 + 2*12 + 2*8 = 52` square feet of wrapping paper plus 6 square feet of slack, for a total of 58 square feet.
> 
> - A present with dimensions `1x1x10` requires `2*1 + 2*10 + 2*10 = 42` square feet of wrapping paper plus 1 square foot of slack, for a total of 43 square feet.
> 
> All numbers in the elves' list are in feet.
> 
> **How many total square feet of wrapping paper should they order?**

Lets see the input:

In [62]:
h 2

3x11x24
13x5x19
1x9x27
24x8x21
6x8x17
19x18x22
10x9x12
12x2x5
26x6x11
9x23x15


I really liked turning the input into an arithmetic expression, so I use that method here too:

In [63]:
h 2 | awk -Fx -vOFS='*2+' '{print $1, $2, $3}'

3*2+11*2+24
13*2+5*2+19
1*2+9*2+27
24*2+8*2+21
6*2+8*2+17
19*2+18*2+22
10*2+9*2+12
12*2+2*2+5
26*2+6*2+11
9*2+23*2+15


I sorted the dimensions to calculate the smallest side:

In [64]:
h 2 | awk -Fx -vOFS='*2+' '{split($0,a);asort(a,s);print $1*$2,$1*$3,$2*$3,s[1]"*"s[2]}'|paste -sd+

33*2+72*2+264*2+3*11+65*2+247*2+95*2+5*13+9*2+27*2+243*2+1*9+192*2+504*2+168*2+8*21+48*2+102*2+136*2+6*8+342*2+418*2+396*2+18*19+90*2+120*2+108*2+9*10+24*2+60*2+10*2+2*5+156*2+286*2+66*2+6*11+207*2+135*2+345*2+9*15


In [65]:
d 2 | awk -Fx -vOFS='*2+' '{split($0,a);asort(a,s);print $1*$2,$1*$3,$2*$3,s[1]*s[2]}'|paste -sd+|bc

1588178


### part2

> The elves are also running low on ribbon.
> Ribbon is all the same width, so they only have to worry about the length they need to order, which they would again like to be exact.
> 
> The ribbon required to wrap a present is the shortest distance around its sides, or the smallest perimeter of any one face.
> Each present also requires a bow made out of ribbon as well; the feet of ribbon required for the perfect bow is equal to the cubic feet of volume of the present.
> Don't ask how they tie the bow, though; they'll never tell.
> 
> For example:
> 
> - A present with dimensions `2x3x4` requires `2+2+3+3 = 10` feet of ribbon to wrap the present plus `2*3*4 = 24` feet of ribbon for the bow, for a total of 34 feet.
> 
> - A present with dimensions `1x1x10` requires `1+1+1+1 = 4` feet of ribbon to wrap the present plus `1*1*10 = 10` feet of ribbon for the bow, for a total of 14 feet.
> 
> **How many total feet of ribbon should they order?**


We can simply turn the input lines into expressions calculating the volume:

In [66]:
h 2 | tr x \*

3*11*24
13*5*19
1*9*27
24*8*21
6*8*17
19*18*22
10*9*12
12*2*5
26*6*11
9*23*15


By using awk to sort the numbers we can calculate half the "shortest distance around the sides". I used `sed p` to duplicate the result lines to have expressions for the .shortest distance around the sides.

In [67]:
h 2 | awk -Fx '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p

3+11
3+11
5+13
5+13
1+9
1+9
8+21
8+21
6+8
6+8
18+19
18+19
9+10
9+10
2+5
2+5
6+11
6+11
9+15
9+15


Can these commands be combined to have the input read only once?

```bash
h 2 | tr x \*
h 2 | awk -Fx '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p
```

Yes, they can: `tee` does just that. `tee` with `>()` process substitution is very useful:

In [68]:
yes A | head -n 3 | tee >(tr A B) >(tr A C)

A
A
A
B
B
B


: 1

From the bash man page:

> Process  substitution allows a process's input or output to be referred
> to using a filename.  It takes the form of  <(list)  or  >(list).   The
> process  list is run asynchronously, and its input or output appears as
> a filename.  This filename is passed as an argument to the current com‐
> mand  as  the  result  of  the expansion.  If the >(list) form is used,
> writing to the file will provide input for list.  If the  <(list)  form
> is  used,  the  file passed as an argument should be read to obtain the
> output of list.  Process substitution is supported on systems that sup‐
> port named pipes (FIFOs) or the /dev/fd method of naming open files.


In [69]:
h 2|tr x \*|tee >(awk -F\* '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p)

3*11*24
13*5*19
1*9*27
24*8*21
6*8*17
19*18*22
10*9*12
12*2*5
26*6*11
9*23*15
3+11
3+11
5+13
5+13
1+9
1+9
8+21
8+21
6+8
6+8
18+19
18+19
9+10
9+10
2+5
2+5
6+11
6+11
9+15
9+15


In [70]:
d 2|tr x \*|tee >(awk -F\* '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p)|paste -sd+|bc

(standard_in) 1: syntax error
(standard_in) 1: syntax error


:( something supposed to work in bash but does not? The problem is probably whitespace:

In [71]:
d 2|tr x \*|tee >(awk -F\* '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p)|paste -sd+|tail -c 60

12+18+12+18+11+20+11+20+3+12+3+12+17+24+17+24+6+11+6+11++++


The last lines were probably empty lines, lets remove them:

In [72]:
d 2|tr x \*|tee >(awk -F\* '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p)|grep .|paste -sd+|tail -c 60

12+18+12+18+11+20+11+20+3+12+3+12+17+24+17+24+6+11+6+11++++


Hmmm... the last lines were probably empty lines or lines with a single byte, lets remove them:

In [73]:
d 2|tr x \*|tee >(awk -F\* '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p)|grep ..|paste -sd+|tail -c 60

+26+12+18+12+18+11+20+11+20+3+12+3+12+17+24+17+24+6+11+6+11


Success:

In [74]:
d 2|tr x \*|tee >(awk -F\* '{split($0,a);asort(a,b);print b[1]"+"b[2]}'|sed p)|grep ..|paste -sd+|bc

3783758


## Day 10: Elves Look, Elves Say

### part1

> Today, the Elves are playing a game called look-and-say.
> They take turns making sequences by reading aloud the previous sequence and using that reading as the next sequence.
> For example, `211` is read as "one two, two ones", which becomes `1221` (1 2, 2 1s).
> 
> Look-and-say sequences are generated iteratively, using the previous value as input for the next step.
> For each step, take the previous value, and replace each run of digits (like `111`) with the number of digits (`3`) followed by the digit itself (`1`).
> 
> For example:
> 
> - `1 becomes `11` (`1` copy of digit `1`).
> 
> - `11` becomes `21` (`2` copies of digit `1`).
> 
> - `21` becomes `1211` (one `2` followed by one `1`).
> 
> - `1211` becomes `111221` (one `1`, one `2`, and two `1`s).
> 
> - `111221` becomes `312211` (three `1`s, two `2`s, and one `1`).
> 
> Starting with the digits in your puzzle input, apply this process 40 times.
> 
> **What is the length of the result?**


In [75]:
d 10

3113322113


In [76]:
d 10 | fold -w1

3
1
1
3
3
2
2
1
1
3


In [77]:
d 10 | fold -w1 | uniq -c 

      1 3
      2 1
      2 3
      2 2
      2 1
      1 3


In [78]:
d 10 | fold -w1 | uniq -c | tr -dc 0-9 

132123222113

In [79]:
yes 'fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)' | head -n 40

fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 | uniq -c | tr -cd 0-9 | tee >(cat)
fold -w1 |

In [80]:
alias yes="yes 2>/dev/null"
eval "echo 3113322113|$(yes 'fold -w1|uniq -c|tr -d "\n "'|head -n40|paste -sd\|)|wc -c"

329356


## Day 6: Probably  a Fire Hazard

> Because your neighbors keep defeating you in the holiday house decorating contest year after year, you've decided to deploy one million lights in a 1000x1000 grid.
> 
> Furthermore, because you've been especially nice this year, Santa has mailed you instructions on how to display the ideal lighting configuration.
> 
> Lights in your grid are numbered from 0 to 999 in each direction; the lights at each corner are at `0,0`, `0,999`, `999,999`, and `999,0`.
> The instructions include whether to `turn on`, `turn off`, or `toggle` various inclusive ranges given as coordinate pairs.
> Each coordinate pair represents opposite corners of a rectangle, inclusive; a coordinate pair like `0,0 through 2,2` therefore refers to 9 lights in a 3x3 square.
> The lights all start turned off.
> 
> To defeat your neighbors this year, all you have to do is set up your lights by doing the instructions Santa sent you in order.
> 
> For example:
> 
> - `turn on 0,0 through 999,999` would turn on (or leave on) every light.
> 
> - `toggle 0,0 through 999,0` would toggle the first line of 1000 lights, turning off the ones that were on, and turning on the ones that were off.
> 
> - `turn off 499,499 through 500,500` would turn off (or leave off) the middle four lights.
> 
> **After following the instructions, how many lights are lit?**

In [81]:
h 6

turn off 660,55 through 986,197
turn off 341,304 through 638,850
turn off 199,133 through 461,193
toggle 322,558 through 977,958
toggle 537,781 through 687,941
turn on 226,196 through 599,390
turn on 240,129 through 703,297
turn on 317,329 through 451,798
turn on 957,736 through 977,890
turn on 263,530 through 559,664


This format looks hard to parse, are the numbers even in the same columns?

In [82]:
h 6 | column -t

turn    off      660,55   through  986,197
turn    off      341,304  through  638,850
turn    off      199,133  through  461,193
toggle  322,558  through  977,958  
toggle  537,781  through  687,941  
turn    on       226,196  through  599,390
turn    on       240,129  through  703,297
turn    on       317,329  through  451,798
turn    on       957,736  through  977,890
turn    on       263,530  through  559,664


Nope. Can it be fixed? Sure, just replace the chars `l` and `,` with spaces:

In [83]:
h 6 | tr l, ' '  | column -t

turn  off  660  55   through  986  197
turn  off  341  304  through  638  850
turn  off  199  133  through  461  193
togg  e    322  558  through  977  958
togg  e    537  781  through  687  941
turn  on   226  196  through  599  390
turn  on   240  129  through  703  297
turn  on   317  329  through  451  798
turn  on   957  736  through  977  890
turn  on   263  530  through  559  664


Ok, so lets create a small, toy example to test our ideas.

    turn on 0,0 through 3,3

     012345
    0XXXX..
    1XXXX..
    2XXXX..
    3XXXX..

    toggle 2,1 through 5,3

     012345
    0XXXX..
    1XX..XX
    2XX..XX
    3XX..XX

    turn off 0,0 through 0,3

     012345
    0.XXX..
    1.X..XX  
    2.X..XX
    3.X..XX
    
    turn on 100,10 throuth 100,10
    
     012345
    0.XXX..
    1.X..XX  plus 100,10
    2.X..XX
    3.X..XX
    
    the result is 13

In [84]:
test_input() {
    echo turn off 0,0, through 2,5
    echo turn on 0,0 through 3,3
    echo toggle 2,1 through 5,3
    echo turn off 0,0 through 0,3
    echo turn on 100,10 through 100,10
}

The plan is:

- find a way to generate all positions in a rectagle given by an "x1,y1 through x2,y2" definition
- use set operations, namely `union`, `diff`, and `xor` to implement `off`, `on`, and `toggle`
- turn every line of the input into one of the set operations and chain them together with pipes

Generating all positions in a rectagle:

In [85]:
echo {008..11}x{099..100} | tr ' ' '\n'

008x099
008x100
009x099
009x100
010x099
010x100
011x099
011x100


Set operations:

In [86]:
# this is a must have when `sort` is used
export LC_ALL=C

# see the respective man page of `comm`
to_set() { tr -s ' \t\n' '\n' | sort -u; }
set_union() { cat "$1" "$2" | to_set; }
set_diff() { comm -23 "$1" "$2" | to_set; }
set_xor() { comm -3 "$1" "$2" | to_set; }

# some tests
i=1
while IFS=@ read -r inp out
do
    res="$(eval "$inp" | paste -sd ' ')"
    echo "test: $inp"
    echo "  exp $out"
    echo "  got $res"
    if test "$res" == "$out"
    then echo "  pass"
    else echo "  fail"
    fi  
done <<\EOF
set_union <(seq 1 5) <(seq 3 8)   @1 2 3 4 5 6 7 8
seq 1 5 | set_union - <(seq 3 8)  @1 2 3 4 5 6 7 8
set_diff <(seq 1 5) <(seq 3 8)    @1 2
seq 1 5 | set_diff - <(seq 3 8)   @1 2
set_xor <(seq 1 5) <(seq 3 8)     @1 2 6 7 8
seq 1 5 | set_xor - <(seq 3 8)    @1 2 6 7 8
EOF

test: set_union <(seq 1 5) <(seq 3 8)   
  exp 1 2 3 4 5 6 7 8
  got 1 2 3 4 5 6 7 8
  pass
test: seq 1 5 | set_union - <(seq 3 8)  
  exp 1 2 3 4 5 6 7 8
  got 1 2 3 4 5 6 7 8
  pass
test: set_diff <(seq 1 5) <(seq 3 8)    
  exp 1 2
  got 1 2
  pass
test: seq 1 5 | set_diff - <(seq 3 8)   
  exp 1 2
  got 1 2
  pass
test: set_xor <(seq 1 5) <(seq 3 8)     
  exp 1 2 6 7 8
  got 1 2 6 7 8
  pass
test: seq 1 5 | set_xor - <(seq 3 8)    
  exp 1 2 6 7 8
  got 1 2 6 7 8
  pass


Use awk to generate the chain. Note that 

- the empty set is simply /dev/null
- brace expansion requires leading zeros otherwise sorting gets messed up
- bash could use more string literals for nesting

In [87]:
test_input | tr l, ' ' | 
awk '
    BEGIN       { print "cat /dev/null" }
    $2 == "off" { printf "set_diff  - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
    $2 == "on"  { printf "set_union - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
    $2 == "e"   { printf "set_xor   - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
'

cat /dev/null
set_diff  - <(echo {0000..0002}x{0000..0005} | tr " " \\n)
set_union - <(echo {0000..0003}x{0000..0003} | tr " " \\n)
set_xor   - <(echo {0002..0005}x{0001..0003} | tr " " \\n)
set_diff  - <(echo {0000..0000}x{0000..0003} | tr " " \\n)
set_union - <(echo {0100..0100}x{0010..0010} | tr " " \\n)


Ok, now we have to eval it somehow. Note that bash can export functions, so that child bash processes can call them. I also inserted logging statements into the pipeline. To get the cardinality of the resulting set, I used `wc -w`.

In [88]:
export -f to_set set_union set_diff set_xor
test_input | tr l, ' ' |
awk '
    BEGIN       { print "cat /dev/null" }
    $2 == "off" { printf "set_diff  - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
    $2 == "on"  { printf "set_union - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
    $2 == "e"   { printf "set_xor   - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
    { printf "tee /tmp/out.%d.log\n", i++ }
' |
paste -sd '|' | 
xargs -0 bash -o pipefail -o errexit -c 'eval "$0"' |
wc -w

13


Yay! It worked!

Lets try with the real input. The solution should be 400410.

In [89]:
if free -g | awk '/Mem:/ {exit $7 < 5}'
then
    export -f to_set set_union set_diff set_xor
    d 6 | tr l, ' ' |
    awk '
        BEGIN       { print "cat /dev/null" }
        $2 == "off" { printf "set_diff  - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
        $2 == "on"  { printf "set_union - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
        $2 == "e"   { printf "set_xor   - <(echo {%04d..%04d}x{%04d..%04d} | tr \" \" \\\\n)\n", $3, $6, $4, $7}
        { printf "tee /tmp/out.%d.log\n", i++ }
    ' |
    paste -sd '|' | 
    xargs -0 bash -o pipefail -o errexit -c 'eval "$0"' |
    wc -w
else
    echo >&2 "Buy more RAM! Forking that so many processes will fail otherwise."
fi

400410


Yey, it works too!

Is it a single line though? Of course, here it is golfed into one line:

In [90]:
if free -g | awk '/Mem:/ {exit $7 < 5}'
then
    d 6|tr l, \ |awk 'BEGIN{print ":"}/of/{printf"comm -23"}/on/{printf"cat"}/to/{printf"comm -3"}{printf " - <(echo {%04d..%d}x{%04d..%d}|tr \" \" \\\\n)|tr -s \"[:space:]\" \\\\n|sort -u\n",$3,$6,$4,$7}'|paste -sd \||xargs -0 bash -c 'eval "$0"'|wc -w
else
    echo >&2 "Buy more RAM! Forking that so many processes will fail otherwise."
fi

400410


## Day 7: Some Assembly Required

### part1

> This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates!
> Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.
> 
> Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from `0` to `65535`).
> A signal is provided to each wire by a gate, another wire, or some specific value.
> Each wire can only get a signal from one source, but can provide its signal to multiple destinations.
> A gate provides no signal until all of its inputs have a signal.
> 
> The included instructions booklet describes how to connect the parts together: `x AND y -> z` means to connect wires `x` and `y` to an `AND` gate, and then connect its output to wire `z`.
> 
> For example:
> 
> - `123 -> x` means that the signal `123` is provided to wire `x`.
> 
> - `x AND y -> z` means that the bitwise `AND` of wire `x` and wire `y` is provided to wire `z`.
> 
> - `p LSHIFT 2 -> q` means that the value from wire `p` is left-shifted by 2 and then provided to wire `q`.
> 
> - `NOT e -> f` means that the bitwise complement of the value from wire `e` is provided to wire `f`.
> 
> Other possible gates include `OR` (bitwise OR) and `RSHIFT` (right-shift).
> If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.
> 
> For example, here is a simple circuit:
> 
> ```
> 123 -> x
> 456 -> y
> x AND y -> d
> x OR y -> e
> x LSHIFT 2 -> f
> y RSHIFT 2 -> g
> NOT x -> h
> NOT y -> i
> ```
> 
> After it is run, these are the signals on the wires:
> 
> ```
> d: 72
> e: 507
> f: 492
> g: 114
> h: 65412
> i: 65079
> x: 123
> y: 456
> ```
> 
> In little Bobby's kit's instructions booklet (provided as your puzzle input), **what signal is ultimately provided to wire a?**


In [91]:
h 7

lf AND lq -> ls
iu RSHIFT 1 -> jn
bo OR bu -> bv
gj RSHIFT 1 -> hc
et RSHIFT 2 -> eu
bv AND bx -> by
is OR it -> iu
b OR n -> o
gf OR ge -> gg
NOT kt -> ku


In [92]:
h 7 | 
awk -F-\> '{print $2"="$1}'

 ls=lf AND lq 
 jn=iu RSHIFT 1 
 bv=bo OR bu 
 hc=gj RSHIFT 1 
 eu=et RSHIFT 2 
 by=bv AND bx 
 iu=is OR it 
 o=b OR n 
 gg=gf OR ge 
 ku=NOT kt 


In [93]:
h 7 |
awk -F-\> '{print $2"="$1}' |
sed '
    s/AND/\&/;
    s/OR/|/;
    s/LSHIFT/<</;
    s/RSHIFT/>>/;
    s/NOT/65535^/;
    s/ //g;
'

ls=lf&lq
jn=iu>>1
bv=bo|bu
hc=gj>>1
eu=et>>2
by=bv&bx
iu=is|it
o=b|n
gg=gf|ge
ku=65535^kt


In [94]:
# convert this to normal notation
h 7 |
awk -F-\> '{print $2"="$1}' |
sed '
    s/AND/\&/;
    s/OR/|/;
    s/LSHIFT/<</;
    s/RSHIFT/>>/;
    s/NOT/65535^/;
    s/ //g;
'

ls=lf&lq
jn=iu>>1
bv=bo|bu
hc=gj>>1
eu=et>>2
by=bv&bx
iu=is|it
o=b|n
gg=gf|ge
ku=65535^kt


In [95]:
# reorder dependency graph into strage format
d 7 |
awk -F-\> '{print $2"="$1}' |
sed '
    s/AND/\&/;
    s/OR/|/;
    s/LSHIFT/<</;
    s/RSHIFT/>>/;
    s/NOT/65535^/;
    s/ //g;
' |
awk -F'[^a-z0-9]' '
    {E[$1]=$0}
    END{for(i in E) {
        split(E[i],A);
        if(E[A[2]]) print E[A[2]], E[i];
        if(E[A[3]]) print E[A[3]], E[i];
    }}
'

ca=bn&by cb=65535^ca
af=y|ae ai=af&ah
ah=65535^ag ai=af&ah
x=v|w aj=x|ai
ai=af&ah aj=x|ai
bz=bn|by cc=bz&cb
cb=65535^ca cc=bz&cb
x=v|w ak=x&ai
ai=af&ah ak=x&ai
cc=bz&cb cd=1&cc
ak=x&ai al=65535^ak
bk=bj|bi ce=bk<<1
aj=x|ai am=aj&al
al=65535^ak am=aj&al
ce=bk<<1 cf=ce|cd
cd=1&cc cf=ce|cd
am=aj&al an=1&am
bn=bl|bm cg=bn>>1
u=t|s ao=u<<1
dy=dw|dx ea=dy>>3
cd=1&cc ch=cd<<15
dy=dw|dx eb=dy>>5
cg=bn>>1 ci=cg|ch
ch=cd<<15 ci=cg|ch
ao=u<<1 ap=ao|an
an=1&am ap=ao|an
ci=cg|ch cj=ci>>2
ea=dy>>3 ec=ea|eb
eb=dy>>5 ec=ea|eb
x=v|w aq=x>>1
an=1&am ar=an<<15
ci=cg|ch ck=ci>>3
ea=dy>>3 ed=ea&eb
eb=dy>>5 ed=ea&eb
ed=ea&eb ee=65535^ed
ci=cg|ch cl=ci>>5
aq=x>>1 as=aq|ar
ar=an<<15 as=aq|ar
as=aq|ar at=as>>2
ck=ci>>3 cm=ck|cl
cl=ci>>5 cm=ck|cl
ec=ea|eb ef=ec&ee
ee=65535^ed ef=ec&ee
as=aq|ar au=as>>3
ck=ci>>3 cn=ck&cl
cl=ci>>5 cn=ck&cl
dz=dy>>2 eg=dz|ef
ef=ec&ee eg=dz|ef
dz=dy>>2 eh=dz&ef
ef=ec&ee eh=dz&ef
fo=fm|fn ga=fo|fz
fz=fw&fy ga=fo|fz
as=aq|ar av=as>>5
cn=ck&cl co=65535^cn
fo=fm|fn gb=fo&fz
fz=fw&fy gb

js=jp>>5 ju=jr&js
lg=lf>>2 ln=lg|lm
lm=lj&ll ln=lg|lm
d=b>>2 l=d&j
j=g&i l=d&j
ju=jr&js jv=65535^ju
lg=lf>>2 lo=lg&lm
lm=lj&ll lo=lg&lm
l=d&j m=65535^l
jt=jr|js jw=jt&jv
jv=65535^ju jw=jt&jv
lo=lg&lm lp=65535^lo
jq=jp>>2 jx=jq|jw
jw=jt&jv jx=jq|jw
ln=lg|lm lq=ln&lp
lp=65535^lo lq=ln&lp
k=d|j n=k&m
m=65535^l n=k&m
jq=jp>>2 jy=jq&jw
jw=jt&jv jy=jq&jw
lf=ld|le lr=lf|lq
lq=ln&lp lr=lf|lq
b=19138 o=b|n
n=k&m o=b|n
jy=jq&jw jz=65535^jy
b=19138 p=b&n
n=k&m p=b&n
lf=ld|le ls=lf&lq
lq=ln&lp ls=lf&lq
p=b&n q=65535^p
ls=lf&lq lt=65535^ls
o=b|n r=o&q
q=65535^p r=o&q
lr=lf|lq lu=lr&lt
lt=65535^ls lu=lr&lt
lu=lr&lt lv=1&lu
r=o&q s=1&r
lc=lb|la lw=lc<<1
c=0 t=c<<1
lw=lc<<1 lx=lw|lv
lv=1&lu lx=lw|lv
t=c<<1 u=t|s
s=1&r u=t|s
lf=ld|le ly=lf>>1
b=19138 v=b>>1
s=1&r w=s<<15
x=v|w aa=x>>5
lv=1&lu lz=lv<<15
z=x>>3 ab=z|aa
aa=x>>5 ab=z|aa
v=b>>1 x=v|w
w=s<<15 x=v|w
z=x>>3 ac=z&aa
aa=x>>5 ac=z&aa
x=v|w y=x>>2
x=v|w z=x>>3
ac=z&aa ad=65535^ac
ab=z|aa ae=ab&ad
ad=65535^ac ae=ab&ad
y=x>>2 af=y|ae
ae=ab&ad af=y|a

In [96]:
# use the magical TSORT binary to apply topological sorting
# "According to its info[2] page, this command was initially written for providing an 
# "ordering of object files that allowed the linker to process them sequentially (each"
# one exactly once, and in order).

d 7 |
awk -F-\> '{print $2"="$1}' |
sed '
    s/AND/\&/;
    s/OR/|/;
    s/LSHIFT/<</;
    s/RSHIFT/>>/;
    s/NOT/65535^/;
    s/ //g;
' |
awk -F'[^a-z0-9]' '
    {E[$1]=$0}
    END{for(i in E) {
        split(E[i],A);
        if(E[A[2]]) print E[A[2]], E[i];
        if(E[A[3]]) print E[A[3]], E[i];
    }}
' |
tsort

b=19138
c=0
v=b>>1
f=b>>5
e=b>>3
d=b>>2
t=c<<1
h=e&f
g=e|f
i=65535^h
j=g&i
l=d&j
k=d|j
m=65535^l
n=k&m
p=b&n
o=b|n
q=65535^p
r=o&q
s=1&r
w=s<<15
u=t|s
x=v|w
ao=u<<1
z=x>>3
y=x>>2
aa=x>>5
aq=x>>1
ac=z&aa
ab=z|aa
ad=65535^ac
ae=ab&ad
ag=y&ae
af=y|ae
ah=65535^ag
ai=af&ah
ak=x&ai
aj=x|ai
al=65535^ak
am=aj&al
an=1&am
ar=an<<15
ap=ao|an
as=aq|ar
bj=ap<<1
bl=as>>1
av=as>>5
au=as>>3
at=as>>2
ax=au&av
aw=au|av
ay=65535^ax
az=aw&ay
bb=at&az
ba=at|az
bc=65535^bb
bd=ba&bc
bf=as&bd
be=as|bd
bg=65535^bf
bh=be&bg
bi=1&bh
bm=bi<<15
bk=bj|bi
bn=bl|bm
ce=bk<<1
bq=bn>>5
bp=bn>>3
bo=bn>>2
cg=bn>>1
bs=bp&bq
br=bp|bq
bt=65535^bs
bu=br&bt
bw=bo&bu
bv=bo|bu
bx=65535^bw
by=bv&bx
ca=bn&by
bz=bn|by
cb=65535^ca
cc=bz&cb
cd=1&cc
ch=cd<<15
cf=ce|cd
ci=cg|ch
cz=cf<<1
db=ci>>1
cl=ci>>5
ck=ci>>3
cj=ci>>2
cn=ck&cl
cm=ck|cl
co=65535^cn
cp=cm&co
cr=cj&cp
cq=cj|cp
cs=65535^cr
ct=cq&cs
cv=ci&ct
cu=ci|ct
cw=65535^cv
cx=cu&cw
cy=1&cx
dc=cy<<15
da=cz|cy
dd=db|dc
du=da<<1
dw=dd>>1
dg=dd>>5
df=dd>>3
de=dd>>2
di=df&dg
dh=df|dg
d

In [97]:
# add a print to the end and execute with python
d 7 |
awk -F-\> '{print $2"="$1}' |
sed '
    s/AND/\&/;
    s/OR/|/;
    s/LSHIFT/<</;
    s/RSHIFT/>>/;
    s/NOT/65535^/;
    s/ //g;
' |
awk -F'[^a-z0-9]' '
    {E[$1]=$0}
    END{for(i in E) {
        split(E[i],A);
        if(E[A[2]]) print E[A[2]], E[i];
        if(E[A[3]]) print E[A[3]], E[i];
    }}
' |
tsort |
sed '$aprint(a)' |
python

  File "<stdin>", line 44
    as=aq|ar
     ^
SyntaxError: invalid syntax


: 1

In [98]:
# no! convert to uppercase with `tr a-z A-Z` first!

In [99]:
# reorder dependency graph into edge-list
d 7|awk -F-\> '{print $2"="$1}'|sed 's/AND/\&/;s/OR/|/;s/LSHIFT/<</;s/RSHIFT/>>/;s/NOT/65535^/;s/ //g;'|
tr a-z A-Z|awk -F'[^A-Z0-9]' '{E[$1]=$0}END{for(i in E) {split(E[i],A);if(E[A[2]]) print E[A[2]], E[i];
if(E[A[3]]) print E[A[3]], E[i];}}'|tsort|sed '$aprint(A)'|python

16076
